# Language Model and Application for Spelling Error Correction
# Objective: Develop a simple English syntax error correction program
Home Exercise:

# a) Improve the model by using interpolation smoothing with the "Stupid Backoff" method (Brants et al., 2007).

+ Step 1: Import the necessary library

In [ ]:
import nltk
import math
import string
import random
import re
import requests
from collections import Counter, defaultdict
from nltk.util import ngrams
from nltk.tokenize import word_tokenize, sent_tokenize
from difflib import get_close_matches
from sklearn.model_selection import train_test_split

nltk.download('punkt')

+ Step 2: Download data

In [ ]:
file_id = "1kuqGxsdtU9EMqDKv3cLEE6r7HIYJyzBw"
url = f"https://drive.google.com/uc?id={file_id}"

response = requests.get(url)
data = response.text 

+ Step 3: Build the preprocess function

In [ ]:
def preprocess_text(text):
    # Step 1: Clean text
    text = text.replace('\n', ' ').strip()
    
    # Step 2: Convert to lowercase
    text = text.lower()
    
    # Step 3: Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Step 4: Tokenization
    tokens = word_tokenize(text)
    
    return tokens

+ Step 4: Build an old model class

In [ ]:
class NGramModel:
    def __init__(self, n):
        self.n = n
        self.ngram_counts = defaultdict(Counter)
        self.context_counts = Counter()
        self.vocab = set()
    
    def train(self, tokenized_corpus):
        for sentence in tokenized_corpus:
            sentence = ['<s>'] * (self.n - 1) + sentence + ['</s>']
            self.vocab.update(sentence)
            ngram_list = list(ngrams(sentence, self.n))
            
            for ngram in ngram_list:
                context = ngram[:-1]
                word = ngram[-1]
                self.ngram_counts[context][word] += 1
                self.context_counts[context] += 1
    
    def probability(self, sentence):
        tokenized_sentence = ['<s>'] * (self.n - 1) + sentence + ['</s>']
        ngram_list = list(ngrams(tokenized_sentence, self.n))
        prob = 1.0
        
        for ngram in ngram_list:
            context = ngram[:-1]
            word = ngram[-1]
            prob *= (self.ngram_counts[context][word] + 1) / (self.context_counts[context] + len(self.vocab))
        
        return prob
    
    def perplexity(self, sentence):
        prob = self.probability(sentence)
        N = len(sentence) + 1  # Including end token
        return math.pow(1/prob, 1/N) if prob > 0 else float('inf')


+ Step 5: Build a model class with smoothing "Stupid Backoff" method

In [ ]:
class NGramModel_with_smoothing:
    def __init__(self, n, alpha=0.4):
        self.n = n  # n-gram size
        self.alpha = alpha  # Stupid Backoff discount factor
        self.ngram_counts = defaultdict(Counter)
        self.context_counts = Counter()
        self.vocab = set()

    def train(self, tokenized_corpus):
        for sentence in tokenized_corpus:
            sentence = ['<s>'] * (self.n - 1) + sentence + ['</s>']
            self.vocab.update(sentence)
            ngram_list = list(ngrams(sentence, self.n))
            for ngram in ngram_list:
                context = ngram[:-1]
                word = ngram[-1]
                self.ngram_counts[context][word] += 1
                self.context_counts[context] += 1

    def probability(self, sentence):
        tokenized_sentence = ['<s>'] * (self.n - 1) + sentence + ['</s>']
        ngram_list = list(ngrams(tokenized_sentence, self.n))
        prob = 1.0
        
        for ngram in ngram_list:
            context = ngram[:-1]
            word = ngram[-1]
            while len(context) > 0:  # Backoff down to unigram
                if context in self.ngram_counts and word in self.ngram_counts[context]:
                    prob *= self.ngram_counts[context][word] / self.context_counts[context]
                    break  # stop when found
                else:
                    prob *= self.alpha  
                    context = context[1:]  

            if len(context) == 0:  
                prob *= (self.ngram_counts[context][word] + 1) / (self.context_counts[context] + len(self.vocab))
                
        return prob

    def perplexity(self, sentence):
        prob = self.probability(sentence)
        N = len(sentence) + 1  # Including end token
        return math.pow(1/max(prob, 1e-40), 1/N)
    
    def generate_next_word(self, context):
        context = tuple(context[-(self.n - 1):]) 

        while len(context) > 0:
            if context in self.ngram_counts:  
                return max(self.ngram_counts[context], key=self.ngram_counts[context].get)  
            context = context[1:]  

        return random.choice(list(self.vocab))  
    
    def correct_spelling(self, word):
        closest_matches = get_close_matches(word, self.vocab, n=1)
        return closest_matches[0] if closest_matches else word
    
    def correct_sentence(self, sentence):
        words = sentence
        corrected_words = []

        for i in range(len(words)):
            word = words[i]

            if word in self.vocab:
                corrected_words.append(word)
                continue

            closest_matches = get_close_matches(word, self.vocab, n=3, cutoff=0.7)

            if not closest_matches:
                corrected_words.append(word)
                continue

            best_word = closest_matches[0]
            max_prob = 0

            for candidate in closest_matches:
                context = tuple(corrected_words[-(self.n - 1):])  
                prob = self.ngram_counts[context][candidate] / self.context_counts[context] if context in self.ngram_counts and candidate in self.ngram_counts[context] else 0

                if prob > max_prob:
                    max_prob = prob
                    best_word = candidate

            corrected_words.append(best_word)

        return " ".join(corrected_words)


In [ ]:
# Load and train model
def load_corpus(data,test_size=0.2, random_state=42):
    
    sentences = sent_tokenize(data)
    tokenized_corpus = [preprocess_text(sent) for sent in sentences]

    train_corpus, test_corpus = train_test_split(tokenized_corpus, test_size=test_size, random_state=random_state)

    return train_corpus, test_corpus

In [ ]:
train_corpus, test_corpus = load_corpus(data)
n1_without_smoothing = NGramModel(1)
n1_without_smoothing.train(train_corpus)
n2_without_smoothing = NGramModel(2)
n2_without_smoothing.train(train_corpus)
n3_without_smoothing = NGramModel(3)
n3_without_smoothing.train(train_corpus)


n1_with_smoothing = NGramModel_with_smoothing(1)
n1_with_smoothing.train(train_corpus)
n2_with_smoothing = NGramModel_with_smoothing(2)
n2_with_smoothing.train(train_corpus)
n3_with_smoothing = NGramModel_with_smoothing(3)
n3_with_smoothing.train(train_corpus)

# b) Compare with the results from In Class Exercise.

+ Take each model result for testing first about a simple sentence

In [ ]:
sentence_analysis = preprocess_text("I like to tell you the tale of one of my favorite projects.")


In [ ]:
print("----------------------OLD MODEL (DO NOT HAVE SMOOTHING)-------------------")
print(f"1-gram probability of sentence1 without smoothing phrase: {n1_without_smoothing.probability(sentence_analysis)}")
print(f"1-gram perplexity of sentence1 without smoothing phrase: {n1_without_smoothing.perplexity(sentence_analysis)}")
print(f"2-gram probability of sentence1 without smoothing phrase: {n2_without_smoothing.probability(sentence_analysis)}")
print(f"2-gram perplexity of sentence1 without smoothing phrase: {n2_without_smoothing.perplexity(sentence_analysis)}")
print(f"3-gram probability of sentence1 without smoothing phrase: {n3_without_smoothing.probability(sentence_analysis)}")
print(f"3-gram perplexity of sentence1 without smoothing phrase: {n3_without_smoothing.perplexity(sentence_analysis)}")

In [ ]:
print("----------------------NEW MODEL (DO HAVE SMOOTHING)-------------------")

print(f"1-gram probability of sentence1 with smoothing phrase: {n1_with_smoothing.probability(sentence_analysis)}")
print(f"1-gram perplexity of sentence1 with smoothing phrase: {n1_with_smoothing.perplexity(sentence_analysis)}")
print(f"2-gram probability of sentence1 with smoothing phrase: {n2_with_smoothing.probability(sentence_analysis)}")
print(f"2-gram perplexity of sentence1 with smoothing phrase: {n2_with_smoothing.perplexity(sentence_analysis)}")
print(f"3-gram probability of sentence1 with smoothing phrase: {n3_with_smoothing.probability(sentence_analysis)}")
print(f"3-gram perplexity of sentence1 with smoothing phrase: {n3_with_smoothing.perplexity(sentence_analysis)}")

# Analyze the result with a sentence example

The comparison between the old model (without smoothing) and the new model (with smoothing) highlights the impact of smoothing, particularly for higher-order n-grams. 

While the 1-gram probabilities and perplexities remain identical in both models, the differences become apparent in 2-gram and 3-gram results. 

The new model significantly increases the probability estimates for 2-grams and 3-grams, which results in much lower perplexities (51.48 vs. 355.64 for 2-grams and 15.60 vs. 1620.35 for 3-grams). 

This suggests that the new model, likely using Stupid Backoff, better handles sparsity issues by assigning nonzero probabilities to unseen n-grams, leading to more stable and reasonable perplexity values. In contrast, the old model, lacking smoothing, suffers from extreme sparsity, making higher-order n-gram probabilities extremely small and their perplexities unrealistically high.

+ We test on the test_dataset

In [ ]:
print("----------------------OLD MODEL (DO NOT HAVE SMOOTHING)-------------------")

for (index,i) in enumerate(test_corpus):
    print(f"INDEX: {index}")
    print(f"1-gram probability of sentence1: {n1_without_smoothing.probability(i)}")
    print(f"1-gram perplexity of sentence1: {n1_without_smoothing.perplexity(i)}")
    print(f"2-gram probability of sentence1: {n2_without_smoothing.probability(i)}")
    print(f"2-gram perplexity of sentence1: {n2_without_smoothing.perplexity(i)}")
    print(f"3-gram probability of sentence1: {n3_without_smoothing.probability(i)}")
    print(f"3-gram perplexity of sentence1: {n3_without_smoothing.perplexity(i)}")
    print()

In [ ]:
print("----------------------NEW MODEL (DO HAVE SMOOTHING)-------------------")

for (index,i) in enumerate(test_corpus):
    print(f"INDEX: {index}")
    print(f"1-gram probability of sentence1: {n1_with_smoothing.probability(i)}")
    print(f"1-gram perplexity of sentence1: {n1_with_smoothing.perplexity(i)}")
    print(f"2-gram probability of sentence1: {n2_with_smoothing.probability(i)}")
    print(f"2-gram perplexity of sentence1: {n2_with_smoothing.perplexity(i)}")
    print(f"3-gram probability of sentence1: {n3_with_smoothing.probability(i)}")
    print(f"3-gram perplexity of sentence1: {n3_with_smoothing.perplexity(i)}")
    print()

**In conclusion, using Stupid Backoff significantly improves the model’s performance by addressing sparsity issues in higher-order n-grams. The new model with smoothing assigns higher probabilities to n-grams, reducing perplexity and making the language model more robust. This improvement is particularly evident in 2-gram and 3-gram cases, where the perplexity drops drastically compared to the unsmoothed model. By incorporating Stupid Backoff, the model better generalizes to unseen data, leading to more stable and realistic probability distributions.**

# c) Use the newly built model to generate the next words for a given word sequence.

+ First, test on some sample.

In [ ]:
sample_sentence_1 = preprocess_text("may")

print(f"1-gram generate_next_word testing:  {n1_with_smoothing.generate_next_word(sample_sentence_1)}")
print(f"2-gram generate_next_word testing:  {n2_with_smoothing.generate_next_word(sample_sentence_1)}")
print(f"3-gram generate_next_word testing:  {n3_with_smoothing.generate_next_word(sample_sentence_1)}")

In [ ]:
sample_sentence_2 = preprocess_text("I like")

print(f"1-gram generate_next_word testing:  {n1_with_smoothing.generate_next_word(sample_sentence_2)}")
print(f"2-gram generate_next_word testing:  {n2_with_smoothing.generate_next_word(sample_sentence_2)}")
print(f"3-gram generate_next_word testing:  {n3_with_smoothing.generate_next_word(sample_sentence_2)}")

In [ ]:
sample_sentence_3 = preprocess_text("the tale of")

print(f"1-gram generate_next_word testing:  {n1_with_smoothing.generate_next_word(sample_sentence_3)}")
print(f"2-gram generate_next_word testing:  {n2_with_smoothing.generate_next_word(sample_sentence_3)}")
print(f"3-gram generate_next_word testing:  {n3_with_smoothing.generate_next_word(sample_sentence_3)}")

+ Test on the test_corpus

In [ ]:
for (index,i) in enumerate(test_corpus):
    print(f"INDEX: {index}")
    print(f"1-gram generate_next_word testing:  {n1_with_smoothing.generate_next_word(i)}")
    print(f"2-gram generate_next_word testing:  {n2_with_smoothing.generate_next_word(i)}")
    print(f"3-gram generate_next_word testing:  {n3_with_smoothing.generate_next_word(i)}")
    print()

# d) Combine with a function that calculates the distance between words to predict the correct word for a misspelled word position. (from difflib import get_close_matches)

# First we test on simple 1 word

+ Test on some simple example

In [ ]:
word_1 = "helo"
print(f"Predict correct word in 1-gram: {n1_with_smoothing.correct_spelling(word_1)}")
print(f"Predict correct word in 2-gram: {n2_with_smoothing.correct_spelling(word_1)}")
print(f"Predict correct word in 3-gram: {n3_with_smoothing.correct_spelling(word_1)}")

In [ ]:
word_2 = "liek"
print(f"Predict correct word in 1-gram: {n1_with_smoothing.correct_spelling(word_2)}")
print(f"Predict correct word in 2-gram: {n2_with_smoothing.correct_spelling(word_2)}")
print(f"Predict correct word in 3-gram: {n3_with_smoothing.correct_spelling(word_2)}")

# Now test on sentences

+ We test on some simple sentence

In [ ]:
sentence_correct_1 = preprocess_text("I liek to bep")
print(f"Predict correct sentence in 1-gram: {n1_with_smoothing.correct_sentence(sentence_correct_1)}")
print(f"Predict correct sentence in 2-gram: {n2_with_smoothing.correct_sentence(sentence_correct_1)}")
print(f"Predict correct sentence in 3-gram: {n3_with_smoothing.correct_sentence(sentence_correct_1)}")

In [ ]:
sentence_correct_2 = preprocess_text("wishs yoau heva a ")
print(f"Predict correct sentence in 1-gram: {n1_with_smoothing.correct_sentence(sentence_correct_2)}")
print(f"Predict correct sentence in 2-gram: {n2_with_smoothing.correct_sentence(sentence_correct_2)}")
print(f"Predict correct sentence in 3-gram: {n3_with_smoothing.correct_sentence(sentence_correct_2)}")

+ Test on test_corpus

In [ ]:
for (index,i) in enumerate(test_corpus):
    print(f"INDEX: {index}")
    print(f"Predict correct sentence in 1-gram: {n1_with_smoothing.correct_sentence(i)}")
    print(f"Predict correct sentence in 2-gram: {n2_with_smoothing.correct_sentence(i)}")
    print(f"Predict correct sentence in 3-gram: {n3_with_smoothing.correct_sentence(i)}")
    print()